In [1]:
import json
from pathlib import Path
from types import SimpleNamespace
import sys
import importlib.resources

In [2]:
sys.path = [
    '/home/marco/Documents/erp/axidoo/libs/ribalta/',
    '/home/marco/Documents/erp/axidoo/odoo/'
] + sys.path
sys.path

['/home/marco/Documents/erp/axidoo/libs/ribalta/',
 '/home/marco/Documents/erp/axidoo/odoo/',
 '/home/marco/Documents/erp/axidoo/libs/ribalta/tests',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/home/marco/.local/share/virtualenvs/axidoo-2AEsS5m1/lib/python3.7/site-packages',
 '/home/marco/.local/share/virtualenvs/axidoo-2AEsS5m1/lib/python3.7/site-packages/IPython/extensions',
 '/home/marco/.ipython']

In [3]:
from mako.template import Template
import dateutil

In [4]:
import ribalta

In [5]:
JSON_DOC = Path('/home/marco/Documents/erp/axidoo/libs/ribalta/tests/data/riba_cred_no_ab.json')
JSON_DOC.exists()

True

In [6]:
CBI_DOC = Path('/home/marco/Documents/erp/axidoo/libs/ribalta/tests/data/riba_out.cbi')
CBI_DOC.exists()

False

In [ ]:
class BankFake:
    def __init__(self, abi, cab, bank_name=''):
        self.bank_name = bank_name
        self.abi = abi
        self.cab = cab
    # end init
# end BankFake

class BankAccountFake:
    def __init__(self, abi, cab, number):
        self.acc_number = number
        self.abi = abi
        self.cab = cab
    # end init
# end BankAccountFake

class InvoiceFake:
    def __init__(self, number, date_invoice):
        self.number = number
        self.date_invoice = dateutil.parser.parse(date_invoice)
    # end init
# end InvoiceFake

class MoveLineFake:
    def __init__(self, date_maturity, amount_residual):
        self.date_maturity = dateutil.parser.parse(date_maturity)
        self.amount_residual = amount_residual
    # end init
# end InvoiceFake

class PartnerFake:
    def __init__(
        self,
        name, ref='',
        vat='', fiscalcode='',
        street='', city='', zip='', state_id=None
    ):
        self.name = name
        self.ref = ref
        self.vat = vat
        self.fiscalcode = fiscalcode
        self.street = street
        self.city = city
        self.zip = zip
        if state_id:
            self.state_id = SimpleNamespace(**state_id)
        else:
            self.state_id = None
        # end if
    # end init
# end PartnerFake

class CompanyFake:
    def __init__(self, partner_id, sia_code):
        self.partner_id = PartnerFake(**partner_id)
        self.sia_code = sia_code
    # end init
# end CompanyFake


class RiBaData:
    def __init__(self, creditor_company, creditor_bank_account, currency_code, lines):
        self.head = {
            'creditor_company': CompanyFake(**creditor_company),
            'creditor_bank_account': BankAccountFake(**creditor_bank_account),
            'currency_code': currency_code,
        }
        self.lines = [
            {
                'debitor_partner': PartnerFake(**line['debitor_partner']),
                'debitor_bank': BankFake(**line['debitor_bank']),
                'invoice': InvoiceFake(**line['invoice']),
                'duedate_move_line': MoveLineFake(**line['duedate_move_line']),
            }
            for line in lines
        ]
    # end __init__
    
    @classmethod
    def build_from_json_doc(cls, jsondoc_path: Path):
        
        with jsondoc_path.open('rb') as json_doc_fp:
            json_dict = json.load(json_doc_fp)
        # end with
        
        new_instance = cls(**json_dict)
        return new_instance
    # end build_from_json_doc
# end RiBaData

In [ ]:
ribadata = RiBaData.build_from_json_doc(JSON_DOC)

In [ ]:
ribadoc = ribalta.Document(**ribadata.head)
for line in ribadata.lines:
    ribadoc.add_line(ribalta.Line(**line))
# end for

In [ ]:
cbi_document = ribadoc.render_cbi()

In [ ]:
cbi_document

In [ ]:
CBI_DOC.write_text(cbi_document)